<div style="float:right; padding-top: 15px; padding-right: 15px">
    <div>
        <a href="https://whiteboxml.com">
            <img src="https://whiteboxml.com/static/img/logo/black_bg_white.svg" width="250">
        </a>
    </div>
</div>

# SQL introduction

## 1. introduction

* acronym for Structured Query Language
* is an ISO standard: https://en.wikipedia.org/wiki/SQL:2016
* extensively used not just by Data Scientists, but to run applications!
* they're based on a branch of algebraic set theory known as relational algebra: https://en.wikipedia.org/wiki/Relational_algebra
* referential integrity, very important, can't be break, as opposite to Big Data systems...
* tools like DBeaver (excellent) to query them, let's install it...
    * https://launchpad.net/~serge-rider/+archive/ubuntu/dbeaver-ce
    * or... `sudo snap install dbeaver-ce`
* integration with jupyter, installing (ipython-sql, sqlalchemy) in kernel environment:
    * https://towardsdatascience.com/jupyter-magics-with-sql-921370099589
    * `pip install ipython-sql` -> not in Jupyter environment, but in your actual environment!

Popular database technologies:
* PostgreSQL
* MySQL
* Oracle RDBMS
* SQL Server
* Others...

## 2. our sample database

let's connect directly using python and dbeaver:

In [1]:
%load_ext sql
%config SqlMagic.autocommit = False

specify path to the database (in this case is just a file .db)

In [2]:
%sql sqlite:///data/chinook.db

let's get tables (this is different for every database technology):

In [3]:
%%sql tables <<

SELECT 
    name
FROM 
    sqlite_master 
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%';

 * sqlite:///data/chinook.db
Done.
Returning data to local variable tables


those tables contains the 'entities in the database', in this case a music store, so there are entities like:
* albums
* artists
* customers
* etc...
every row in a table is an entity, which have certain attributes, represented by the columns of the table.

In [10]:
tables

[('albums',),
 ('artists',),
 ('customers',),
 ('employees',),
 ('genres',),
 ('invoices',),
 ('invoice_items',),
 ('media_types',),
 ('playlists',),
 ('playlist_track',),
 ('tracks',)]

### 3. simple select query

for example, for albums columns (first select query here)...

In [11]:
%%sql albums <<

SELECT *
FROM albums
LIMIT 10;

 * sqlite:///data/chinook.db
Done.
Returning data to local variable albums


In [12]:
albums

[(1, 'For Those About To Rock We Salute You', 1),
 (2, 'Balls to the Wall', 2),
 (3, 'Restless and Wild', 2),
 (4, 'Let There Be Rock', 1),
 (5, 'Big Ones', 3),
 (6, 'Jagged Little Pill', 4),
 (7, 'Facelift', 5),
 (8, 'Warner 25 Anos', 6),
 (9, 'Plays Metallica By Four Cellos', 7),
 (10, 'Audioslave', 8)]

## 4. relationships between tables

the interesting thing about relational databases is that they have referential integrity as relations among entities are enforced, check in dbeaver...

![relations](relations_diagram.png "Entity Relations diagram")

more info about entity-relationship diagrams: https://dbeaver.com/docs/wiki/Database-Structure-Diagrams/

### types of relationships

#### one-to-one

![one-to-one](one_to_one.gif "one-to-many relationship")

other examples:
* one-to-one: marriage

#### one-to-many

![one-to-many](one_to_many.gif "one-to-one relationship")

other examples:
* many to many: like tracks and palylists


#### many-to-many

![many-to-many](many_to_many.gif "many-to-many relationship")

other examples:
* one to many: like a customer with many invoices, but an invoice can only have one customer

## 5. data normalization 

* relationships are enforced through foreign keys, columns in tables referencing elements in a foreign ('other') table...
* a primary key is defined for every entity, and must be unique...
* this way of data storage avoids repeated data (it database is well designed, not the most usual case :-D)

## 6. database design

* database design is an art, improved with experience. it is not the job of a DS, DA, but an application architect
* this about entities and try to think what kind of relationships exists among them

## 7. bonus, sqlalchemy and reading sql data from pandas, real examples...

In [16]:
import pandas as pd
import sqlalchemy
import sqlite3

### sqlite3 standard

In [17]:
cnx = sqlite3.connect('./data/chinook.db')

In [18]:
cnx.execute("select * from invoices").fetchall()

[(1,
  2,
  '2009-01-01 00:00:00',
  'Theodor-Heuss-Straße 34',
  'Stuttgart',
  None,
  'Germany',
  '70174',
  1.98),
 (2,
  4,
  '2009-01-02 00:00:00',
  'Ullevålsveien 14',
  'Oslo',
  None,
  'Norway',
  '0171',
  3.96),
 (3,
  8,
  '2009-01-03 00:00:00',
  'Grétrystraat 63',
  'Brussels',
  None,
  'Belgium',
  '1000',
  5.94),
 (4,
  14,
  '2009-01-06 00:00:00',
  '8210 111 ST NW',
  'Edmonton',
  'AB',
  'Canada',
  'T6G 2C7',
  8.91),
 (5,
  23,
  '2009-01-11 00:00:00',
  '69 Salem Street',
  'Boston',
  'MA',
  'USA',
  '2113',
  13.86),
 (6,
  37,
  '2009-01-19 00:00:00',
  'Berger Straße 10',
  'Frankfurt',
  None,
  'Germany',
  '60316',
  0.99),
 (7,
  38,
  '2009-02-01 00:00:00',
  'Barbarossastraße 19',
  'Berlin',
  None,
  'Germany',
  '10779',
  1.98),
 (8,
  40,
  '2009-02-01 00:00:00',
  '8, Rue Hanovre',
  'Paris',
  None,
  'France',
  '75002',
  1.98),
 (9,
  42,
  '2009-02-02 00:00:00',
  '9, Place Louis Barthou',
  'Bordeaux',
  None,
  'France',
  '33000',
  

### pandas

In [19]:
enn = sqlalchemy.create_engine('sqlite:///data/chinook.db')

In [20]:
invoices = pd.read_sql_table(table_name='invoices', con=enn)

In [21]:
invoices.groupby('BillingCountry')['Total'].sum().sort_values(ascending=False)

BillingCountry
USA               523.06
Canada            303.96
France            195.10
Brazil            190.10
Germany           156.48
United Kingdom    112.86
Czech Republic     90.24
Portugal           77.24
India              75.26
Chile              46.62
Ireland            45.62
Hungary            45.62
Austria            42.62
Finland            41.62
Netherlands        40.62
Norway             39.62
Sweden             38.62
Australia          37.62
Denmark            37.62
Poland             37.62
Italy              37.62
Argentina          37.62
Spain              37.62
Belgium            37.62
Name: Total, dtype: float64

### sql alchemy

In [22]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

In [23]:
Base = declarative_base()

class Article(Base):
    __tablename__ = 'articles'

    id = Column(Integer, primary_key=True)
    title = Column(String)
    url = Column(String)

    def __repr__(self):
        return f"<Article(title={self.title}, url={self.url})>"

In [24]:
engine = create_engine('sqlite:///data/test_database.db')
Session = sessionmaker(bind=engine)
Base.metadata.create_all(engine)  # will not create already existent tables
session = Session()

In [25]:
article = Article(title='The Definitive Data Scientist Environment Setup', 
                  url='https://davidadrian.cc/definitive-data-scientist-setup/')

In [26]:
session.add(article)
session.commit()

In [27]:
published = session.query(Article).all()

In [28]:
published

[<Article(title=The Definitive Data Scientist Environment Setup, url=https://davidadrian.cc/definitive-data-scientist-setup/)>]

In [29]:
session.close()

<div style="padding-top: 25px; float: right">
    <div>    
        <i>&nbsp;&nbsp;© Copyright by</i>
    </div>
    <div>
        <a href="https://whiteboxml.com">
            <img src="https://whiteboxml.com/static/img/logo/black_bg_white.svg" width="125">
        </a>
    </div>
</div>